# OpenSearch: семантическое чанкирование и индекс `makar_ozon_semantic`

Этот ноутбук:
- берёт markdown‑документы из папки `Документация 2`,
- режет их на предложения и объединяет предложения в чанки по косинусному сходству эмбеддингов,
- считает эмбеддинги через Yandex textEmbedding API,
- создаёт индекс `makar_ozon_semantic` и индексирует в него все чанки.

Индекс потом можно использовать в сервисе `/search`, `/rag/answer` и в ноутбуке `opensearch_eval_colbert.ipynb` (через `index_name="makar_ozon_semantic"`).


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import re
import json
from typing import List, Dict, Any
from dotenv import load_dotenv

import numpy as np
import requests
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests.auth import HTTPBasicAuth

load_dotenv()

MD_DIR = "/Users/admin/Downloads/Telegram Desktop/Документация 2"
INDEX_NAME = "makar_ozon_semantic"

OPENSEARCH_URL = os.getenv("OPENSEARCH_URL")
OPENSEARCH_USER = os.getenv("OPENSEARCH_USER")
OPENSEARCH_PASSWORD = os.getenv("OPENSEARCH_PASSWORD")

if not OPENSEARCH_URL:
    raise ValueError("OPENSEARCH_URL must be set in environment variables")
if not OPENSEARCH_USER:
    raise ValueError("OPENSEARCH_USER must be set in environment variables")
if not OPENSEARCH_PASSWORD:
    raise ValueError("OPENSEARCH_PASSWORD must be set in environment variables")

YANDEX_API_KEY = os.getenv("YANDEX_API_KEY")
YANDEX_FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")

if not YANDEX_API_KEY:
    raise ValueError("YANDEX_API_KEY must be set in environment variables")
if not YANDEX_FOLDER_ID:
    raise ValueError("YANDEX_FOLDER_ID must be set in environment variables")
YANDEX_EMBED_MODEL = os.getenv("YANDEX_EMBED_MODEL", "text-search-doc")
YANDEX_EMBEDDINGS_URL = os.getenv(
    "YANDEX_EMBEDDINGS_URL",
    "https://llm.api.cloud.yandex.net/foundationModels/v1/textEmbedding",
)

SIM_THRESHOLD = float(os.getenv("SEMANTIC_SIM_THRESHOLD", "0.8"))
MAX_SENT_PER_CHUNK = int(os.getenv("MAX_SENT_PER_CHUNK", "8"))

print("OpenSearch:", OPENSEARCH_URL, "index:", INDEX_NAME)
print("MD_DIR:", MD_DIR)
print("Yandex folder:", YANDEX_FOLDER_ID)
print("Yandex embed model:", YANDEX_EMBED_MODEL)

OpenSearch: https://localhost:9200 index: makar_ozon_semantic
MD_DIR: /Users/admin/Downloads/Telegram Desktop/Документация 2
Yandex folder: b1gql4st0j9joerfcttt
Yandex embed model: text-search-doc


In [27]:
def create_client(url: str, user: str, password: str) -> OpenSearch:
    auth = HTTPBasicAuth(user, password) if user and password else None
    client = OpenSearch(
        hosts=[url],
        http_compress=True,
        http_auth=auth,
        use_ssl=url.startswith("https://"),
        verify_certs=False,
        connection_class=RequestsHttpConnection,
        timeout=60,
        max_retries=3,
        retry_on_timeout=True,
    )
    return client


client = create_client(OPENSEARCH_URL, OPENSEARCH_USER, OPENSEARCH_PASSWORD)
print("OpenSearch client ready")

OpenSearch client ready


/Users/admin/Library/Python/3.8/lib/python/site-packages/opensearchpy/connection/http_requests.py:157: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [ ]:
import os

if not (YANDEX_API_KEY and YANDEX_FOLDER_ID):
    raise ValueError(
        "YANDEX_API_KEY and YANDEX_FOLDER_ID must be set in environment variables"
    )

MODEL_URI = f"emb://{YANDEX_FOLDER_ID}/{YANDEX_EMBED_MODEL}/latest"


def yandex_embed_one(text: str) -> List[float]:
    body = {"modelUri": MODEL_URI, "text": text}
    headers = {
        "Authorization": f"Api-Key {YANDEX_API_KEY}",
        "x-folder-id": YANDEX_FOLDER_ID,
        "Content-Type": "application/json",
    }
    resp = requests.post(YANDEX_EMBEDDINGS_URL, headers=headers, json=body, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    emb = data.get("embedding") or (data.get("result") or {}).get("embedding")
    if emb is None:
        raise RuntimeError(f"Bad embedding response: {data}")
    return emb


_test_vec = yandex_embed_one("тестовая строка для определения размерности")
EMBED_DIM = len(_test_vec)
print("Yandex embedding dim:", EMBED_DIM)

Yandex embedding dim: 256


In [ ]:
INDEX_BODY: Dict[str, Any] = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 0,
            "knn": True,
            "knn.algo_param.ef_search": 100,
            "similarity": {
                "custom_similarity": {
                    "type": "BM25",
                    "k1": 1.2,
                    "b": 0.75,
                    "discount_overlaps": "true",
                }
            },
            "analysis": {
                "filter": {
                    "russian_stemmer": {"type": "stemmer", "language": "russian"},
                    "unique_pos": {"type": "unique", "only_on_same_position": False},
                    "my_multiplexer": {
                        "type": "multiplexer",
                        "filters": [
                            "keyword_repeat",
                            "russian_stemmer",
                            "remove_duplicates",
                        ],
                    },
                },
                "analyzer": {
                    "search_text_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase", "my_multiplexer", "unique_pos"],
                        "char_filter": ["e_mapping"],
                    },
                    "ru_international_translit_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "russian_stemmer",
                        ],
                        "char_filter": ["transliteration_filter", "e_mapping"],
                    },
                    "text_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "russian_stemmer",
                        ],
                        "char_filter": ["e_mapping"],
                    },
                    "exact_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase"],
                        "char_filter": ["e_mapping"],
                    },
                    "text_standard": {"type": "standard"},
                    "text_whitespace": {"type": "whitespace"},
                    "text_lowercase": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase"],
                    },
                },
                "char_filter": {
                    "transliteration_filter": {
                        "type": "mapping",
                        "mappings": [
                            "a => а",
                            "b => б",
                            "v => в",
                            "g => г",
                            "d => д",
                            "e => е",
                            "ye => ё",
                            "zh => ж",
                            "z => з",
                            "i => и",
                            "j => й",
                            "k => к",
                            "l => л",
                            "m => м",
                            "n => н",
                            "o => о",
                            "p => п",
                        ],
                    },
                    "e_mapping": {"type": "mapping", "mappings": ["e => ё"]},
                },
            },
        }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "analyzer": "text_analyzer",
                "similarity": "BM25",
            },
            "source": {"type": "keyword"},
            "chunk_id": {"type": "keyword"},
            "text_vector": {
                "type": "knn_vector",
                "dimension": EMBED_DIM,
                "space_type": "cosinesimil",
                "method": {
                    "name": "hnsw",
                    "engine": "faiss",
                    "parameters": {"ef_construction": 512, "m": 64},
                },
            },
        }
    },
}

if client.indices.exists(index=INDEX_NAME):
    print(f"Index {INDEX_NAME} exists. Deleting...")
    client.indices.delete(index=INDEX_NAME)

print(f"Creating index {INDEX_NAME}...")
client.indices.create(index=INDEX_NAME, body=INDEX_BODY)
print("Index created")

Index makar_ozon_semantic exists. Deleting...
Creating index makar_ozon_semantic...
Index created


/Users/admin/Library/Python/3.8/lib/python/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/admin/Library/Python/3.8/lib/python/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/admin/Library/Python/3.8/lib/python/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
SENT_SPLIT_REGEX = re.compile(r"([.!?]+)\s+")


def split_into_sentences(text: str) -> List[str]:
    text = text.strip()
    if not text:
        return []
    parts = SENT_SPLIT_REGEX.split(text)
    sentences: List[str] = []
    buf = ""
    for part in parts:
        if not part:
            continue
        if SENT_SPLIT_REGEX.match(part):
            buf += part + " "
            sentences.append(buf.strip())
            buf = ""
        else:
            buf += part + " "
    if buf.strip():
        sentences.append(buf.strip())
    return [s for s in sentences if s.strip()]


def build_semantic_chunks_for_file(path: str, fname: str) -> List[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    paragraphs = [p.strip() for p in content.split("\n\n") if p.strip()]
    sentences: List[str] = []
    for p in paragraphs:
        sentences.extend(split_into_sentences(p))

    if not sentences:
        return []

    sent_embs = []
    for s in sentences:
        sent_embs.append(np.array(yandex_embed_one(s), dtype="float32"))
    sent_embs = np.stack(sent_embs, axis=0)

    chunk_spans: List[tuple] = []

    cur_indices: List[int] = [0]
    cur_vec = sent_embs[0].copy()

    for i in range(1, len(sentences)):
        vec = sent_embs[i]
        num = float(np.dot(cur_vec, vec))
        den = float(np.linalg.norm(cur_vec) * np.linalg.norm(vec))
        sim = num / den if den != 0.0 else 0.0

        if sim >= SIM_THRESHOLD and len(cur_indices) < MAX_SENT_PER_CHUNK:
            cur_indices.append(i)
            cur_vec = sent_embs[cur_indices].mean(axis=0)
        else:
            start, end = cur_indices[0], cur_indices[-1]
            chunk_spans.append((start, end))
            cur_indices = [i]
            cur_vec = vec.copy()

    if cur_indices:
        start, end = cur_indices[0], cur_indices[-1]
        chunk_spans.append((start, end))

    merged_spans: List[tuple] = []
    i = 0
    while i < len(chunk_spans):
        start, end = chunk_spans[i]
        sent_count = end - start + 1

        if sent_count < 3 and i + 1 < len(chunk_spans):
            next_start, next_end = chunk_spans[i + 1]
            merged_spans.append((start, next_end))
            i += 2
        else:
            merged_spans.append((start, end))
            i += 1

    if len(merged_spans) >= 2:
        last_start, last_end = merged_spans[-1]
        if (last_end - last_start + 1) < 3:
            prev_start, prev_end = merged_spans[-2]
            merged_spans[-2] = (prev_start, last_end)
            merged_spans.pop()

    chunks: List[Dict[str, Any]] = []
    for start, end in merged_spans:
        idxs = list(range(start, end + 1))
        text = " ".join(sentences[j] for j in idxs)
        vec = sent_embs[idxs].mean(axis=0)
        chunk_id = f"{fname}::s{start}-{end}"
        chunks.append(
            {
                "text": text,
                "source": fname,
                "chunk_id": chunk_id,
                "text_vector": vec.tolist(),
            }
        )

    return chunks

In [ ]:
if not os.path.isdir(MD_DIR):
    raise FileNotFoundError(f"Markdown dir not found: {MD_DIR}")

md_files = [fn for fn in os.listdir(MD_DIR) if fn.lower().endswith(".md")]
md_files.sort()

docs: List[Dict[str, Any]] = []
for fname in md_files:
    path = os.path.join(MD_DIR, fname)
    file_chunks = build_semantic_chunks_for_file(path, fname)
    print(f"{fname}: {len(file_chunks)} semantic chunks")
    docs.extend(file_chunks)

print("Total semantic chunks:", len(docs))

Банковские карты и операции.md: 4 semantic chunks
Валютные операции и счета.md: 2 semantic chunks
Вклады и сберегательные продукты.md: 3 semantic chunks
Жалобы и эскалация инцидентов.md: 4 semantic chunks
Зарплатные проекты и массовые выплаты.md: 3 semantic chunks
Идентификация клиента и безопасность операций.md: 2 semantic chunks
Ипотечные кредиты и залоговая недвижимость.md: 1 semantic chunks
Кредиты и реструктуризация задолженности.md: 3 semantic chunks
Мобильный банк.md: 2 semantic chunks
Мошенничество и подозрительные операции.md: 3 semantic chunks
Обслуживание малого бизнеса и расчётно‑кассовое обслуживание.md: 3 semantic chunks
Обслуживание через банкоматы и терминалы самообслуживания.md: 1 semantic chunks
Платежи и переводы.md: 1 semantic chunks
интернет-банк.md: 3 semantic chunks
Total semantic chunks: 35


In [23]:
docs

[{'text': '# Банковские карты и операции: руководство для операторов колл‑центра Документ описывает порядок работы операторов колл‑центра с обращениями, связанными с банковскими картами и операциями по ним, и фиксирует единый подход к консультированию клиентов в этой области . Основная задача оператора — обеспечить корректные, единообразные и безопасные ответы по всем вопросам, связанным с картами, начиная от их выпуска и активации и заканчивая закрытием, спорными операциями и общением в сложных ситуациях . Правильная работа с картами напрямую влияет на финансовую безопасность клиентов и репутацию банка, поэтому любые действия в этой сфере должны опираться на чётко описанные процедуры и аккуратную коммуникацию. В повседневной работе оператору приходится иметь дело с разными видами карт: дебетовыми, кредитными, виртуальными, премиальными . Важно понимать ключевые отличия этих продуктов . Дебетовая карта даёт доступ к собственным средствам клиента и, как правило, не допускает уход в мину

In [ ]:
BULK_ENDPOINT = f"/{INDEX_NAME}/_bulk"
lines: List[str] = []

for i, d in enumerate(docs):
    doc_id = d.get("chunk_id") or f"doc-{i}"
    meta = {"index": {"_index": INDEX_NAME, "_id": doc_id}}
    src = {
        "text": d["text"],
        "source": d["source"],
        "chunk_id": d["chunk_id"],
        "text_vector": d["text_vector"],
    }
    lines.append(json.dumps(meta, ensure_ascii=False))
    lines.append(json.dumps(src, ensure_ascii=False))

payload = "\n".join(lines) + "\n"
resp = client.transport.perform_request("POST", BULK_ENDPOINT, body=payload)
if isinstance(resp, dict) and resp.get("errors"):
    errs = sum(
        1 for it in resp.get("items", []) if (it.get("index") or {}).get("error")
    )
    print("Bulk completed with errors:", errs)
else:
    print(f"Bulk indexed {len(docs)} semantic chunks into '{INDEX_NAME}'")

Bulk indexed 35 semantic chunks into 'makar_ozon_semantic'


/Users/admin/Library/Python/3.8/lib/python/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
import random

OUT_QUERIES_PATH = "test_queries_semantic.json"
N_QUESTIONS = 100

YANDEX_LLM_MODEL = os.getenv("YANDEX_LLM_MODEL", "yandexgpt-lite")
YANDEX_COMPLETION_URL = os.getenv(
    "YANDEX_COMPLETION_URL",
    "https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
)

if not (YANDEX_API_KEY and YANDEX_FOLDER_ID):
    raise ValueError(
        "YANDEX_API_KEY and YANDEX_FOLDER_ID must be set in environment variables"
    )

llm_headers = {
    "Authorization": f"Api-Key {YANDEX_API_KEY}",
    "x-folder-id": YANDEX_FOLDER_ID,
    "Content-Type": "application/json",
}


def generate_question_for_chunk(text: str) -> str:
    """Генерирует один естественный пользовательский вопрос по смыслу данного чанка."""
    snippet = text.strip().replace("\n", " ")
    if len(snippet) > 1200:
        snippet = snippet[:1200]

    prompt = (
        "Ты помогаешь составлять тестовые пользовательские запросы по текстам.\n"
        "Ниже дан фрагмент текста (чанк). Сформулируй ОДИН естественный, "
        "конкретный вопрос на русском языке, который пользователь мог бы задать, "
        "чтобы найти именно этот фрагмент. Не пиши ничего, кроме самого вопроса.\n\n"
        f"Текст чанка:\n{snippet}\n\n"
        "Вопрос:"
    )

    body = {
        "modelUri": f"gpt://{YANDEX_FOLDER_ID}/{YANDEX_LLM_MODEL}/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.4,
            "maxTokens": 120,
        },
        "messages": [
            {"role": "user", "text": prompt},
        ],
    }

    resp = requests.post(
        YANDEX_COMPLETION_URL,
        headers=llm_headers,
        json=body,
        timeout=60,
    )
    resp.raise_for_status()
    data = resp.json()

    try:
        text_out = data["result"]["alternatives"][0]["message"]["text"].strip()
    except Exception as e:
        raise RuntimeError(f"Bad completion response: {data}") from e

    return text_out


if not docs:
    raise RuntimeError(
        "Список docs пуст — сначала собери семантические чанки (ячейки выше)"
    )

num_available = len(docs)
num_to_sample = min(N_QUESTIONS, num_available)
all_indices = list(range(num_available))
random.shuffle(all_indices)
sample_indices = all_indices[:num_to_sample]

print(
    f"Всего чанков: {num_available}. Будем генерировать вопросы для {num_to_sample} случайных чанков."
)

results: List[Dict[str, Any]] = []
for idx in sample_indices:
    d = docs[idx]
    text = (d.get("text") or "").strip()
    if not text:
        continue

    print(f"[{idx}] generating question…")
    try:
        q = generate_question_for_chunk(text)
    except Exception as e:  # noqa: BLE001
        print(f"  error on chunk {idx}: {e}")
        continue

    results.append(
        {
            "id": idx,
            "question": q,
            "chunk_text": text,
            "source": d.get("source"),
            "chunk_id": d.get("chunk_id"),
        }
    )

print(f"Generated {len(results)} query–chunk pairs")

with open(OUT_QUERIES_PATH, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Saved to {OUT_QUERIES_PATH}")

Всего чанков: 35. Будем генерировать вопросы для 35 случайных чанков.
[17] generating question…
[26] generating question…
[16] generating question…
[21] generating question…
[4] generating question…
[1] generating question…
[23] generating question…
[19] generating question…
[27] generating question…
[10] generating question…
[22] generating question…
[33] generating question…
[9] generating question…
[7] generating question…
[32] generating question…
[8] generating question…
[5] generating question…
[25] generating question…
[14] generating question…
[31] generating question…
[28] generating question…
[6] generating question…
[13] generating question…
[18] generating question…
[20] generating question…
[24] generating question…
[15] generating question…
[29] generating question…
[2] generating question…
[0] generating question…
[34] generating question…
[12] generating question…
[3] generating question…
[11] generating question…
[30] generating question…
Generated 35 query–chunk pairs